In [1]:
import openai
from openai import OpenAI
import json

import base64
import requests

import pandas as pd
import numpy as np

import uuid
import os
import tiktoken
import time
#openai clip model
#from transformers import CLIPProcessor, CLIPModel


import cloudinary
import cloudinary.uploader
import cloudinary.api

import weaviate
import weaviate.util

from PIL import Image
import io
from io import BytesIO
import matplotlib.pyplot as plt

### Reading descriptions dataset

In [2]:
# pd.set_option('display.max_colwidth',None)

In [3]:
# descrip_df = pd.DataFrame(columns=["Case_ID", "First_Party", "Second_Party", "Image_ID"])
# descrip_df.head()

In [4]:
descrip_df = pd.read_csv("data\descriptions.csv")
embedding_df = pd.read_csv("data\descriptions_With_Embeddings.csv")

In [5]:
descrip_df.head(1)

,Case_ID,First_Party,Second_Party,Image_ID
0,74604c28-43cd-49d4-9f54-489e19321aad,كنت أقود سيارتي البيضاء بهدوء في خطي، وفجأة تو...,كنت أقود سيارتي الفضية بهدوء، وعلى الرغم من أن...,data/imgs/47.jpeg


### Starting connection with openai API

In [6]:
#saving the api key
api_key = "sk-proj-UxsihIsXy5aq4Dq3G2DuT3BlbkFJwOAd6S4hvjcZN3nsCKvO"
openai.api_key = api_key

#### Get the descriptions by sending the image to chat-gpt 4o

In [7]:
#encoding function to encode the images before sending it to the API
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [7]:
#get the responses from the API and store them in the responses list, and save each image path in a list as well
images_folder = "data/imgs"
responses = []
images_paths = []


for image_name in os.listdir(images_folder):
  #image path
  image_path = os.path.join(images_folder, image_name)
  #store the image in the images_paths list
  images_paths.append(image_path)

  #encoded image
  base64_image = encode_image(image_path)

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            #prompt
            "text": """[no prose]
                      [Output Only JSON]
                      I have car crash image I want you to take place as the effected parities, where your will write a description of the first party side, then write the other party description of their perspective. 
                      and do it in 4 scenarios 
                      1st: both parties say the truth.
                      2nd: both parties laying
                      3rd: first party is laying.
                      4th: second party is laying
                      (all in arabic)
                      
                      so end result will be in json format:
                      {
                        scenario_n:{

                          first_party:

                          second_party:
                        },
                      }
                      """
          },
          {
            #include the image
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    #number of tokens
    "max_tokens": 650
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  print("---------------------I AM HERE---------------------")
  print(response.json())
  #store the response in responses list
  responses.append(response.json())
  
  print("---------------------I AM HERE2---------------------")

  # #get the descriptions content from the response
  # response_dict = response.json()
  # #extract the content
  # content = response_dict['choices'][0]['message']['content']

  # #clean the content string then turn it into json 
  # content = content.replace("```json", "").replace("```", "").strip()
  # # content = content.replace('",', '"')
  # descrip_json = json.loads(content)

  # #Store the descriptions in the dataframe
  # case_id = uuid.uuid4()

  # for i in range(1, 5):
  #     data = {
  #         "Case_ID": case_id,
  #         "First_Party": descrip_json[f'scenario_{i}']['first_party'],
  #         "Second_Party": descrip_json[f'scenario_{i}']['second_party'],
  #         "Image_ID": image_path
  #     }

  #     #create new row 
  #     new_row_df = pd.DataFrame([data])
  #     #add the new row to the descrip dataframe
  #     descrip_df = pd.concat([descrip_df, new_row_df], ignore_index=True)


  # descrip_df.head(20)

---------------------I AM HERE---------------------
{'id': 'chatcmpl-9YAnjTWGU0p6m7z9lDGcbztQTUvVw', 'object': 'chat.completion', 'created': 1717931699, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "scenario_1": {\n    "first_party": "كنت أقود سيارتي البيضاء بهدوء في خطي، وفجأة توقف السائق أمامي فجأة. حاولت التوقف بسرعة لكنني اصطدمت به من الخلف.",\n    "second_party": "كنت أقود سيارتي الفضية بهدوء، وعلى الرغم من أنني توقفت بسبب ازدحام مفاجئ، إلا أن السيارة التي خلفي لم تتوقف في الوقت المناسب واصطدمت بي من الخلف."\n  },\n  "scenario_2": {\n    "first_party": "كنت أقود بحذر وعلى السرعة القانونية، وفجأة توقفت السيارة أمامي دون أي سبب واضح؛ لم أتمكن من التوقف في الوقت المناسب واصطدمت بها.",\n    "second_party": "كنت أتبع جميع قواعد المرور ولم أقم بأي توقف مفاجئ. السيارة التي خلفي كانت تسير بسرعة كبيرة واصطدمت بي من الخلف."\n  },\n  "scenario_3": {\n    "first_party": "كنت أسير بسرعة القانون وعلى بعد مسافة آمنة من السيارة 

In [8]:
#Store the responses and images id's in the dataframe
for r, img in zip(responses, images_paths):
    #get the descriptions content from the response
    response_dict = r
    #extract the content
    content = response_dict['choices'][0]['message']['content']

    #clean the content string then turn it into json 
    content = content.replace("```json",'').replace("```",'').strip()
    # content = content.replace('",', '"')
    
    descrip_json = json.loads(content)

    if isinstance(descrip_json, list):
        continue

    #Store the descriptions in the dataframe
    case_id = uuid.uuid4()

    for i in range(1, 5):
        data = {
            "Case_ID": case_id,
            "First_Party": descrip_json[f'scenario_{i}']['first_party'],
            "Second_Party": descrip_json[f'scenario_{i}']['second_party'],
            "Image_ID": img
        }

        #create new row 
        new_row_df = pd.DataFrame([data])
        #add the new row to the descrip dataframe
        descrip_df = pd.concat([descrip_df, new_row_df], ignore_index=True) 

In [16]:
descrip_df.shape

(372, 4)

In [24]:
# descrip_df.to_csv("data/descriptions.csv", index=False)

### Get the fault assessment based on the generated descriptions using chat-gpt 4o model

# حمد شغلك هنا

In [25]:
# test_image = "data\imgs\\18.jpeg"

# test_image_bytes = io.BytesIO()
# test_image.save(test_image_bytes, format=format)
# base64_image = base64.b64encode(test_image_bytes.getvalue()).decode(
#         "utf-8")

test_image = "data\imgs\\67.jpeg"
test_image_b64 = encode_image(test_image)



Assistant_ID = "asst_SmTRNw48MnpChRfXFe05EGXm"
client = OpenAI(api_key= api_key)


In [26]:
# Configuration
cloudinary.config(
  cloud_name = 'dxz1w6uho',
  api_key = '634863842981523',
  api_secret = '7GwzeMp2rrnlO-s1I0_zJoriFJs'
)

In [31]:
# Uploading the image
response = cloudinary.uploader.upload(
    "data\imgs\\67.jpeg",
    quality="auto:best",  # Adjusts quality automatically to the best
    fetch_format="auto"  # Adjusts format automatically
)

In [32]:
# Getting the URL
img_url = response['url']
print(f"Image URL: {img_url}")

Image URL: http://res.cloudinary.com/dxz1w6uho/image/upload/v1718051048/draty8ba9rdtsjols711.jpg


In [33]:

thread = client.beta.threads.create(
    messages=[
        {
           "role": "user",
           "content": [
          {
            "type": "text",
            #prompt
            "text": """
                          first_party:
كنت أقود السيارة الحمراء بسرعتي الطبيعية عندما توقفت فجأة السيارة الصفراء أمامي دون إشارة. لم أستطع التوقف في الوقت المناسب واصطدمت بها.
                          
                          second_party:
كنت أقود السيارة الصفراء وأبطأت بسبب ازدحام المرور المفاجئ. فجأة شعرت بضربة قوية من الخلف والسيارة الحمراء اصطدمت بي.                          
                      """
            },
            
           {
            "type": "image_url",
            "image_url": {
                "url": img_url,
                "detail": "high"
              }
           }
           ]            
         }
    ],
)


In [42]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=Assistant_ID,
)

print(f" Run Created: {run.id}")

while (run.status != "completed"):
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print("Run status: {run.status}")
    time.sleep(1)
else:
    print("run compleated")


messages_response = client.beta.threads.messages.list(
    thread_id=thread.id
)
messages = messages_response.data


Latest_messages = messages[0]
print(f"Response {Latest_messages.content[0].text.value}")

 Run Created: run_7bYAyz5L5y1OKexMSBNar8sN
Run status: {run.status}
Run status: {run.status}
Run status: {run.status}
Run status: {run.status}
Run status: {run.status}
Run status: {run.status}
Run status: {run.status}
run compleated
Response ```json
{
  "First_Party_Fault": 100,
  "Second_Party_Fault": 0,
  "Justification": "السيارة الحمراء مسؤولة عن الحادث، لأنها لم تحافظ على مسافة الأمان الكافية مع السيارة التي أمامها، مخالفة بذلك المادة ٥٠/٢/١٩ من أنظمة المرور السعودية【4:11†المصدر】."
}
```


### Embedding descriptions

In [123]:
# Create the new 'descriptions' column
descrip_df['Descriptions'] = 'First_Party:' + descrip_df['First_Party'] + '\nSecond_Party:' + descrip_df['Second_Party']

In [124]:
descrip_df.head(5)

,Case_ID,First_Party,Second_Party,Image_ID,Descriptions
0,74604c28-43cd-49d4-9f54-489e19321aad,كنت أقود سيارتي البيضاء بهدوء في خطي، وفجأة توقف السائق أمامي فجأة. حاولت التوقف بسرعة لكنني اصطدمت به من الخلف.,كنت أقود سيارتي الفضية بهدوء، وعلى الرغم من أنني توقفت بسبب ازدحام مفاجئ، إلا أن السيارة التي خلفي لم تتوقف في الوقت المناسب واصطدمت بي من الخلف.,data/imgs/47.jpeg,First_Party:كنت أقود سيارتي البيضاء بهدوء في خطي، وفجأة توقف السائق أمامي فجأة. حاولت التوقف بسرعة لكنني اصطدمت به من الخلف.\nSecond_Party:كنت أقود سيارتي الفضية بهدوء، وعلى الرغم من أنني توقفت بسبب ازدحام مفاجئ، إلا أن السيارة التي خلفي لم تتوقف في الوقت المناسب واصطدمت بي من الخلف.
1,74604c28-43cd-49d4-9f54-489e19321aad,كنت أقود بحذر وعلى السرعة القانونية، وفجأة توقفت السيارة أمامي دون أي سبب واضح؛ لم أتمكن من التوقف في الوقت المناسب واصطدمت بها.,كنت أتبع جميع قواعد المرور ولم أقم بأي توقف مفاجئ. السيارة التي خلفي كانت تسير بسرعة كبيرة واصطدمت بي من الخلف.,data/imgs/47.jpeg,First_Party:كنت أقود بحذر وعلى السرعة القانونية، وفجأة توقفت السيارة أمامي دون أي سبب واضح؛ لم أتمكن من التوقف في الوقت المناسب واصطدمت بها.\nSecond_Party:كنت أتبع جميع قواعد المرور ولم أقم بأي توقف مفاجئ. السيارة التي خلفي كانت تسير بسرعة كبيرة واصطدمت بي من الخلف.
2,74604c28-43cd-49d4-9f54-489e19321aad,كنت أسير بسرعة القانون وعلى بعد مسافة آمنة من السيارة أمامي، لكن فجأة توقفت فجأة واصطدمت بها عن غير قصد.,كنت أسير ببطء بسبب الازدحام، وفجأة شعرت بضربة خلفي. بدى لي أن السيارة التي خلفي لم تكن منتبهة ولم تستطع التوقف في الوقت المناسب.,data/imgs/47.jpeg,First_Party:كنت أسير بسرعة القانون وعلى بعد مسافة آمنة من السيارة أمامي، لكن فجأة توقفت فجأة واصطدمت بها عن غير قصد.\nSecond_Party:كنت أسير ببطء بسبب الازدحام، وفجأة شعرت بضربة خلفي. بدى لي أن السيارة التي خلفي لم تكن منتبهة ولم تستطع التوقف في الوقت المناسب.
3,74604c28-43cd-49d4-9f54-489e19321aad,كنت أقود على السرعة القانونية وفجأة توقفت السيارة الفضية أمامي بدون أي تحذير، لم أتمكن من تجنب الاصطدام.,كنت أقود ببطء في خطي، وفجأة جاءت السيارة البيضاء بسرعة واخترقت الازدحام واصطدمت بي من الخلف دون أن تحاول حتى التوقف.,data/imgs/47.jpeg,First_Party:كنت أقود على السرعة القانونية وفجأة توقفت السيارة الفضية أمامي بدون أي تحذير، لم أتمكن من تجنب الاصطدام.\nSecond_Party:كنت أقود ببطء في خطي، وفجأة جاءت السيارة البيضاء بسرعة واخترقت الازدحام واصطدمت بي من الخلف دون أن تحاول حتى التوقف.
4,8d860487-2bd9-4560-b327-4af7cd1e9c2a,كنت أسير على الطريق الرئيسي بسرعة محددة وبانتظام. كان الضوء الأخضر لي، فجأة ظهرت سيارة أخرى من الجانب دون أي إنذار وتجاوزت الإشارة الحمراء واصطدمت بي.,كنت أقترب من التقاطع وقررت المرور معتقداً أن بإمكاني تجاوز الوقت المتبقي للإشارة الحمراء. لم أتحقق من السيارات القادمة من الجهة الأخرى واصطدمت بسيارة كانت تسير على الطريق الرئيسي.,data/imgs/10.jpeg,First_Party:كنت أسير على الطريق الرئيسي بسرعة محددة وبانتظام. كان الضوء الأخضر لي، فجأة ظهرت سيارة أخرى من الجانب دون أي إنذار وتجاوزت الإشارة الحمراء واصطدمت بي.\nSecond_Party:كنت أقترب من التقاطع وقررت المرور معتقداً أن بإمكاني تجاوز الوقت المتبقي للإشارة الحمراء. لم أتحقق من السيارات القادمة من الجهة الأخرى واصطدمت بسيارة كانت تسير على الطريق الرئيسي.


In [125]:
descrip_df.shape

(300, 5)

In [126]:
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191
# top_n = 1000   # subsample to 1k most recent Descrptions and remove samples that are too long
min_length = 10  # remove samples that are too short
max_length = 100  # remove samples that are too long

In [127]:
encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
descrip_df["n_tokens"] = descrip_df.Descriptions.apply(lambda x: len(encoding.encode(x)))
len(descrip_df)

300

In [128]:
descrip_df.head(2)

,Case_ID,First_Party,Second_Party,Image_ID,Descriptions,n_tokens
0,74604c28-43cd-49d4-9f54-489e19321aad,كنت أقود سيارتي البيضاء بهدوء في خطي، وفجأة توقف السائق أمامي فجأة. حاولت التوقف بسرعة لكنني اصطدمت به من الخلف.,كنت أقود سيارتي الفضية بهدوء، وعلى الرغم من أنني توقفت بسبب ازدحام مفاجئ، إلا أن السيارة التي خلفي لم تتوقف في الوقت المناسب واصطدمت بي من الخلف.,data/imgs/47.jpeg,First_Party:كنت أقود سيارتي البيضاء بهدوء في خطي، وفجأة توقف السائق أمامي فجأة. حاولت التوقف بسرعة لكنني اصطدمت به من الخلف.\nSecond_Party:كنت أقود سيارتي الفضية بهدوء، وعلى الرغم من أنني توقفت بسبب ازدحام مفاجئ، إلا أن السيارة التي خلفي لم تتوقف في الوقت المناسب واصطدمت بي من الخلف.,188
1,74604c28-43cd-49d4-9f54-489e19321aad,كنت أقود بحذر وعلى السرعة القانونية، وفجأة توقفت السيارة أمامي دون أي سبب واضح؛ لم أتمكن من التوقف في الوقت المناسب واصطدمت بها.,كنت أتبع جميع قواعد المرور ولم أقم بأي توقف مفاجئ. السيارة التي خلفي كانت تسير بسرعة كبيرة واصطدمت بي من الخلف.,data/imgs/47.jpeg,First_Party:كنت أقود بحذر وعلى السرعة القانونية، وفجأة توقفت السيارة أمامي دون أي سبب واضح؛ لم أتمكن من التوقف في الوقت المناسب واصطدمت بها.\nSecond_Party:كنت أتبع جميع قواعد المرور ولم أقم بأي توقف مفاجئ. السيارة التي خلفي كانت تسير بسرعة كبيرة واصطدمت بي من الخلف.,177
2,74604c28-43cd-49d4-9f54-489e19321aad,كنت أسير بسرعة القانون وعلى بعد مسافة آمنة من السيارة أمامي، لكن فجأة توقفت فجأة واصطدمت بها عن غير قصد.,كنت أسير ببطء بسبب الازدحام، وفجأة شعرت بضربة خلفي. بدى لي أن السيارة التي خلفي لم تكن منتبهة ولم تستطع التوقف في الوقت المناسب.,data/imgs/47.jpeg,First_Party:كنت أسير بسرعة القانون وعلى بعد مسافة آمنة من السيارة أمامي، لكن فجأة توقفت فجأة واصطدمت بها عن غير قصد.\nSecond_Party:كنت أسير ببطء بسبب الازدحام، وفجأة شعرت بضربة خلفي. بدى لي أن السيارة التي خلفي لم تكن منتبهة ولم تستطع التوقف في الوقت المناسب.,174
3,74604c28-43cd-49d4-9f54-489e19321aad,كنت أقود على السرعة القانونية وفجأة توقفت السيارة الفضية أمامي بدون أي تحذير، لم أتمكن من تجنب الاصطدام.,كنت أقود ببطء في خطي، وفجأة جاءت السيارة البيضاء بسرعة واخترقت الازدحام واصطدمت بي من الخلف دون أن تحاول حتى التوقف.,data/imgs/47.jpeg,First_Party:كنت أقود على السرعة القانونية وفجأة توقفت السيارة الفضية أمامي بدون أي تحذير، لم أتمكن من تجنب الاصطدام.\nSecond_Party:كنت أقود ببطء في خطي، وفجأة جاءت السيارة البيضاء بسرعة واخترقت الازدحام واصطدمت بي من الخلف دون أن تحاول حتى التوقف.,163
4,8d860487-2bd9-4560-b327-4af7cd1e9c2a,كنت أسير على الطريق الرئيسي بسرعة محددة وبانتظام. كان الضوء الأخضر لي، فجأة ظهرت سيارة أخرى من الجانب دون أي إنذار وتجاوزت الإشارة الحمراء واصطدمت بي.,كنت أقترب من التقاطع وقررت المرور معتقداً أن بإمكاني تجاوز الوقت المتبقي للإشارة الحمراء. لم أتحقق من السيارات القادمة من الجهة الأخرى واصطدمت بسيارة كانت تسير على الطريق الرئيسي.,data/imgs/10.jpeg,First_Party:كنت أسير على الطريق الرئيسي بسرعة محددة وبانتظام. كان الضوء الأخضر لي، فجأة ظهرت سيارة أخرى من الجانب دون أي إنذار وتجاوزت الإشارة الحمراء واصطدمت بي.\nSecond_Party:كنت أقترب من التقاطع وقررت المرور معتقداً أن بإمكاني تجاوز الوقت المتبقي للإشارة الحمراء. لم أتحقق من السيارات القادمة من الجهة الأخرى واصطدمت بسيارة كانت تسير على الطريق الرئيسي.,243


In [130]:
api_key

'sk-proj-UxsihIsXy5aq4Dq3G2DuT3BlbkFJwOAd6S4hvjcZN3nsCKvO'

In [134]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

def get_embedding(text, model):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [136]:
# This may take a few minutes
descrip_df["embedding"] = descrip_df.Descriptions.apply(lambda x: get_embedding(x, model=embedding_model))
descrip_df.to_csv("data/descriptions_With_Embeddings.csv")

In [137]:
descrip_df.head(1)

,Case_ID,First_Party,Second_Party,Image_ID,Descriptions,n_tokens,embedding
0,74604c28-43cd-49d4-9f54-489e19321aad,كنت أقود سيارتي البيضاء بهدوء في خطي، وفجأة توقف السائق أمامي فجأة. حاولت التوقف بسرعة لكنني اصطدمت به من الخلف.,كنت أقود سيارتي الفضية بهدوء، وعلى الرغم من أنني توقفت بسبب ازدحام مفاجئ، إلا أن السيارة التي خلفي لم تتوقف في الوقت المناسب واصطدمت بي من الخلف.,data/imgs/47.jpeg,First_Party:كنت أقود سيارتي البيضاء بهدوء في خطي، وفجأة توقف السائق أمامي فجأة. حاولت التوقف بسرعة لكنني اصطدمت به من الخلف.\nSecond_Party:كنت أقود سيارتي الفضية بهدوء، وعلى الرغم من أنني توقفت بسبب ازدحام مفاجئ، إلا أن السيارة التي خلفي لم تتوقف في الوقت المناسب واصطدمت بي من الخلف.,188,"[-0.02193339169025421, -0.012562095187604427, 0.005614063236862421, -0.026451455429196358, -0.03348998352885246, 0.012997813522815704, -0.011328676715493202, -0.016289163380861282, -0.0022623836994171143, -0.030031049624085426, 0.011945386417210102, 0.038450468331575394, 0.0029377471655607224, -0.011114169843494892, 0.02981654182076454, -0.010832628235220909, 0.030915891751646996, -0.00019031255214940757, 0.0122403334826231, -0.008747884072363377, -0.01371507253497839, 0.01140911690890789, -0.007601609453558922, 0.006046430207788944, 0.0074876523576676846, 0.02871719002723694, 0.01518981158733368, 0.0010675099911168218, -0.001219173427671194, -0.014479256235063076, 0.024373413994908333, -0.008459639735519886, -0.012823525816202164, 0.011911869049072266, -0.020485466346144676, -0.020619532093405724, 0.008258538320660591, -0.008948984555900097, 0.027564212679862976, -0.01934589445590973, -0.002597551792860031, 0.014693763107061386, -0.005198454950004816, 0.02783234789967537, 0.0031237653456628323, 0.024802429601550102, -0.02407846599817276, 0.028368616476655006, -0.023059556260704994, 0.044617559760808945, -0.0019657600205391645, -0.010959992185235023, -0.02788597345352173, -0.003225991502404213, 0.004712461493909359, 0.0018149345414713025, 0.0013289409689605236, 0.025218037888407707, -0.008017217740416527, -0.014318374916911125, -0.0049604857340455055, 0.020847447216510773, 0.0005831922753714025, -0.007132374215871096, -0.004994002636522055, 0.00945843942463398, 0.00037036059075035155, -0.012602315284311771, -0.0222417451441288, 0.01848786510527134, 0.018702372908592224, 0.01811247691512108, 0.008010514080524445, 0.001685056951828301, 0.023260656744241714, -0.014747390523552895, -0.007507762406021357, 0.012099563144147396, -0.00015491043450310826, -3.4590379073051736e-05, 0.014988711103796959, -0.010061741806566715, -0.013326277956366539, 0.02493649534881115, -0.0010867820819839835, 0.0006879322463646531, 9.133326966548339e-05, 0.01889006607234478, -0.007661939598619938, -0.02218811959028244, 0.012495061382651329, 0.02267076075077057, 0.01435859501361847, -0.008178098127245903, -0.005369390826672316, 0.023515384644269943, -0.0011747636599466205, 0.0352596677839756, 0.007199407555162907, -0.022308779880404472, ...]"
1,74604c28-43cd-49d4-9f54-489e19321aad,كنت أقود بحذر وعلى السرعة القانونية، وفجأة توقفت السيارة أمامي دون أي سبب واضح؛ لم أتمكن من التوقف في الوقت المناسب واصطدمت بها.,كنت أتبع جميع قواعد المرور ولم أقم بأي توقف مفاجئ. السيارة التي خلفي كانت تسير بسرعة كبيرة واصطدمت بي من الخلف.,data/imgs/47.jpeg,First_Party:كنت أقود بحذر وعلى السرعة القانونية، وفجأة توقفت السيارة أمامي دون أي سبب واضح؛ لم أتمكن من التوقف في الوقت المناسب واصطدمت بها.\nSecond_Party:كنت أتبع جميع قواعد المرور ولم أقم بأي توقف مفاجئ. السيارة التي خلفي كانت تسير بسرعة كبيرة واصطدمت بي من الخلف.,177,"[-0.017143549397587776, -0.014932338148355484, 0.0038296584971249104, -0.03266199305653572, -0.028505979105830193, 0.006690246984362602, -0.005754478275775909, -0.012894293293356895, -0.006373883690685034, -0.02473626285791397, 0.013760128989815712, 0.028212927281856537, 0.009037993848323822, -0.00813219603151083, 0.019301477819681168, -0.013347192667424679, 0.03271527215838432, -0.012974216602742672, 0.021699177101254463, -0.002783995121717453, -0.0009724002447910607, 0.0025025985669344664

### Using Weaviate with clip model 

In [ ]:
# Connect to your Weaviate instance
client = weaviate.Client("http://localhost:8080")

#delete the old schema
# client.schema.delete_class("CarCrash")

#define the schema
schema = {
    "class": "carCrash",
    "description": "Images of different car crashes with their descriptions",
    "moduleConfig": {
        "multi2vec-clip": {
            "imageFields": ["carCrash_image"],
            "textFields": ["descriptions"],
            "weights": {
                "textFields": [0.5],
                "imageFields": [0.5]
            }
        }
    },
    "vectorIndexType": "hnsw",
    "vectorizer": "multi2vec-clip",
    "properties": [
        {
            "dataType": ["string"],
            "name": "descriptions"
        },
        {
            "dataType": ["blob"],
            "name": "carCrash_image"
        }
    ]
}

client.schema.create_class(schema)

In [ ]:
def add_car_crash(image_path, description):
    img_base64 = weaviate.util.image_encoder_b64(image_path)
    
    properties = {
        "carCrash_image": img_base64,
        "descriptions": description
    }

    client.data_object.create(
        data_object=properties,
        class_name="CarCrash"
    )


In [ ]:
descrips_list:list[str]= embedding_df['Descriptions']
images_paths_list:list[str] = embedding_df['Image_ID']

In [ ]:
for img, descrip in zip(images_paths_list, descrips_list):
    add_car_crash(img, descrip)
    

In [ ]:
def test_hybrid(near_text, near_image, client):
    near_text = {"concepts": [near_text]}
    near_image = {"image": near_image}

    res = client.query.get("carCrash",
                           ["carCrash_image",
                            "descriptions",
                            "_additional {certainty}"]
                          ).with_hybrid(
                              query=near_text,
                              alpha=0.5
                          ).with_near_image(near_image).do()
    
    return res['data']['Get']['carCrash']


In [ ]:
def testText(text, client):
    nearText = {
        "concepts": [text]
    }
    res = client.query.get("CarCrash",
                           ["carCrash_image",
                            "descriptions",
                            "_additional {certainty}"]
                          ).with_near_text(nearText).do()
    
    return res['data']['Get']['CarCrash']

In [ ]:
def testImage(nearImage, client):
    nearImage = {
        "image": nearImage
    }
    imres = client.query.get("CarCrash",
                             ["carCrash_image",
                              "descriptions",
                              "_additional {certainty}"]
                             ).with_near_image(nearImage).do()
    
    return imres['data']['Get']['CarCrash']

In [ ]:
test_description = """First_Party:كنت أقود سيارتي البيضاء بهدوء في خطي، وفجأة توقف السائق أمامي فجأة. حاولت التوقف بسرعة لكنني اصطدمت به من الخلف.
Second_Party:كنت أقود سيارتي الفضية بهدوء، وعلى الرغم من أنني توقفت بسبب ازدحام مفاجئ، إلا أن السيارة التي خلفي لم تتوقف في الوقت المناسب واصطدمت بي من الخلف."""
test_image = "data/imgs/47.jpeg"


# result = test_hybrid(test_description, test_image, client)
# display(result)


text_results = testText(test_description, client)
image_results = testImage(test_image, client)

In [ ]:
print(image_results)

In [ ]:
print(text_results)

In [ ]:
mg = weaviate.util.image_decoder_b64(text_results[0]['carCrash_image'])
image = Image.open(BytesIO(mg))

plt.imshow(image)
plt.axis('off')  # Hide the axis
plt.show()

In [ ]:
#Esrra's schema
class_obj = {
        "class": "Item",
            "description": "A class to implement CLIP example",
            "moduleConfig": {
            "multi2vec-clip": {
              "imageFields": [
                  "image"
              ],
              "textFields": [
                  "text"
              ],
              "weights": {
                "textFields": [0.7],
                "imageFields": [0.3]
              }
            }
          },
            "vectorIndexType": "hnsw",
            "vectorizer": "multi2vec-clip",
            "properties": [
                {
                "dataType": [
                    "string"
                ],
                "name": "text"
                },
                {
                "dataType": [
                    "blob"
                ],
                "name": "image"
                }
            ]
        }


In [ ]:
def weaviate_img_search(img_str, text):
    sourceImage = { "image": img_str}

    weaviate_results = client.query.get(
        "CarCrash", "descriptions"
        ).with_near_image(
            sourceImage, encode= False
        ).do()
    return weaviate_results['data']['Get']['CarCrash']

In [ ]:
res = weaviate_img_search(test_image, test_description)
print(res)